In [1]:
from lightfm import LightFM
from lightfm.datasets import fetch_movielens
from lightfm.evaluation import precision_at_k, auc_score, reciprocal_rank
import numpy as np
from pyarrow.parquet import ParquetFile
import pandas as pd
from scipy.sparse import coo_matrix
from tqdm import tqdm
import dask
from dask.distributed import Client
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k, auc_score
import time
import dask.dataframe as dd
from dask_jobqueue import SLURMCluster
import os

/ext3/miniconda3/lib/python3.9/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


In [2]:
# Set LOCAL to True for single-machine execution while developing
# Set LOCAL to False for cluster execution
LOCAL = True

if LOCAL:
    # This line creates a single-machine dask client
    client = Client()
else:    
    # This line creates a SLURM cluster dask and dask client
    # Logging outputs will be stored in /scratch/{your-netid}
    
    cluster = SLURMCluster(
                           # Memory and core limits should be sufficient here
                           memory='4GB', cores=2,

                           # Ensure that Dask uses the correct version of Python on the cluster
                           python='/scratch/work/public/dask/{}/bin/python'.format(dask.__version__),                           
                           
                           # Place the output logs in an accessible location
                           job_extra=['--export=NONE --output=/scratch/{}/slurm-%j.out'.format(os.environ['SLURM_JOB_USER'])]
    )

    cluster.submit_command = 'slurm'
    cluster.scale(50)

    display(cluster)
    client = Client(cluster)

display(client)

distributed.diskutils - INFO - Found stale lock file and directory '/home/yl5308/final-project-group-34/dask-worker-space/worker-v07mquhn', purging


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 1
Total threads: 1,Total memory: 32.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:34835,Workers: 1
Dashboard: http://127.0.0.1:8787/status,Total threads: 1
Started: Just now,Total memory: 32.00 GiB
Comm: tcp://127.0.0.1:39055,Total threads: 1
Dashboard: http://127.0.0.1:36949/status,Memory: 32.00 GiB
Nanny: tcp://127.0.0.1:35143,


In [3]:
# def average_precision(model_future, ground_truth, user_id, k):
#     scores = model_future.predict(user_id, ground_truth)
#     top_k = np.argsort(-scores)[:k]
#     hits = np.isin(top_k, np.arange(len(ground_truth)))
    
#     if np.sum(hits) == 0:
#         return 0.0
    
#     precisions = np.cumsum(hits) / (np.arange(k) + 1)
#     return np.sum(precisions * hits) / np.min([k, len(ground_truth)])


# def mean_average_precision(model_future, ground_truths, k):
#     ground_truths = ground_truths.tocsr()
    
#     # Create a list of tasks
#     tasks = [dask.delayed(average_precision)(model_future, ground_truths[user_id].indices, user_id, k) for user_id in tqdm(range(model.user_embeddings.shape[0]))]
    
#     # Compute average precisions in parallel
#     aps = client.compute(tasks, sync=True)
    
#     return np.mean(aps)

In [4]:
file_path1 = "/scratch/work/courses/DSGA1004-2021/listenbrainz/interactions_train_small.parquet"
file_path2 = "/scratch/work/courses/DSGA1004-2021/listenbrainz/tracks_train.parquet"
file_path3 = "/scratch/work/courses/DSGA1004-2021/listenbrainz/interactions_test.parquet"
interaction_t = dd.read_parquet(file_path1)
track = dd.read_parquet(file_path2)
interaction_v = dd.read_parquet(file_path3)

In [5]:
interaction_t = interaction_t.head(n=10000000, npartitions=-1)

/ext3/miniconda3/lib/python3.9/site-packages/dask/sizeof.py:151: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, col in df.iteritems():


In [6]:
interaction_v = interaction_v.head(n=2500000, npartitions=-1)

/ext3/miniconda3/lib/python3.9/site-packages/dask/sizeof.py:151: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, col in df.iteritems():


In [7]:
track = track.head(n=10000000, npartitions=-1)

/ext3/miniconda3/lib/python3.9/site-packages/dask/sizeof.py:151: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, col in df.iteritems():


In [8]:
interaction_t = dd.from_pandas(interaction_t,npartitions = 100)
interaction_v = dd.from_pandas(interaction_v,npartitions = 100)
track = dd.from_pandas(track,npartitions = 100)

In [9]:
track['recording_mbid'] = track['recording_mbid'].fillna(track['recording_msid'])

In [10]:
print(type(track['recording_mbid']))

<class 'dask.dataframe.core.Series'>


In [11]:
track['rn'] = track['recording_mbid'].map_partitions(pd.Series.rank,method = 'dense')

In [12]:
# track.isnull().any().compute()

In [13]:
# Merge dataframes
data_t = track.merge(interaction_t, on='recording_msid',how = 'inner')

In [14]:
data_t.dtypes

recording_msid            object
artist_name               object
track_name                object
recording_mbid            object
rn                       float64
user_id                    Int32
timestamp         datetime64[ns]
dtype: object

In [15]:
# Fill NA values, drop unnecessary columns, and rename columns
data_t['recording_mbid'] = data_t['recording_mbid'].fillna(data_t['recording_msid'])

In [16]:
data_t.dtypes

recording_msid            object
artist_name               object
track_name                object
recording_mbid            object
rn                       float64
user_id                    Int32
timestamp         datetime64[ns]
dtype: object

In [17]:
data_t = data_t.drop(columns=["artist_name", "track_name","timestamp","recording_msid", "recording_mbid"])

In [18]:
data_t.columns

Index(['rn', 'user_id'], dtype='object')

In [19]:
# data_t.head()

In [20]:
data_t = data_t.rename(columns={'rn': 'recording_mbid'})

In [21]:
# data_t['recording_mbid'].isnull().any().compute()

In [22]:
# Compute popularity
beta = 10000
popularity = data_t.groupby('recording_mbid').apply(lambda df: len(df['user_id'].unique()) / (len(df) + beta), meta=('popularity', 'float64')).reset_index()

In [23]:
# popularity.head()

In [24]:
popularity = popularity.rename(columns = {'index':'recording_mbid'})

In [25]:
popularity["recording_mbid"] = popularity["recording_mbid"].astype("float64")

In [26]:
data_t.dtypes

recording_mbid    float64
user_id             Int32
dtype: object

In [27]:
popularity.dtypes

recording_mbid    float64
popularity        float64
dtype: object

In [28]:
popularity = popularity.merge(data_t, on = "recording_mbid")

In [29]:
data_v = track.merge(interaction_v, on='recording_msid')
data_v['recording_mbid'] = data_v['recording_mbid'].fillna(data_v['recording_msid'])

In [30]:

data_v = data_v.drop(columns=["artist_name", "track_name","timestamp","recording_msid", "recording_mbid"])
data_v = data_v.rename(columns={'rn': 'recording_mbid'})

In [31]:
# data_t[data_t.isnull()].head()

In [32]:
# data_v.head()

In [33]:
track.columns

Index(['recording_msid', 'artist_name', 'track_name', 'recording_mbid', 'rn'], dtype='object')

In [34]:
data = Dataset()
user_id = dd.multi.concat([data_t['user_id'],data_v['user_id']]).unique().astype('int64')
recording_mbid = track['rn'].unique().astype('int64')
data.fit(recording_mbid,user_id)

/ext3/miniconda3/lib/python3.9/site-packages/dask/sizeof.py:151: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, col in df.iteritems():
/ext3/miniconda3/lib/python3.9/site-packages/dask/sizeof.py:151: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, col in df.iteritems():
/ext3/miniconda3/lib/python3.9/site-packages/dask/sizeof.py:151: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, col in df.iteritems():
/ext3/miniconda3/lib/python3.9/site-packages/dask/sizeof.py:151: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, col in df.iteritems():
/ext3/miniconda3/lib/python3.9/site-packages/dask/sizeof.py:151: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, col in df.iteritems():


In [35]:
# data_t['user_id'].head(10)

In [36]:
type(data_v)

dask.dataframe.core.DataFrame

In [37]:
train_values = data_t.compute().values
test_values = data_v.compute().values

/ext3/miniconda3/lib/python3.9/site-packages/dask/sizeof.py:151: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, col in df.iteritems():
/ext3/miniconda3/lib/python3.9/site-packages/dask/sizeof.py:151: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, col in df.iteritems():
/ext3/miniconda3/lib/python3.9/site-packages/dask/sizeof.py:151: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, col in df.iteritems():
/ext3/miniconda3/lib/python3.9/site-packages/dask/sizeof.py:151: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, col in df.iteritems():
/ext3/miniconda3/lib/python3.9/site-packages/dask/sizeof.py:151: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for name, col in df.iteritems():


In [38]:
del data_t
del data_v

In [39]:
train, weights_matrix = data.build_interactions([tuple(i) for i in train_values])
test, weights_matrix = data.build_interactions([tuple(i) for i in test_values])

In [40]:
# Instantiate and train the model
start_time = time.time()
model = LightFM(loss='warp')
model.fit(train, epochs=50, num_threads=2)
end_time = time.time()

In [41]:
print("Total LightFM model training time: {} seconds".format(end_time - start_time))

Total LightFM model training time: 196.9637234210968 seconds


In [42]:
training_auc = auc_score(model, train).mean()

print("Training Accuracy : {}".format(training_auc))

# Evaluate the trained model
test_precision = precision_at_k(model, test, k=100).mean()
test_auc = auc_score(model, test).mean()

print("Testing Precision: {}".format(test_precision))
print("Testing Accuracy: {}".format(test_auc))

Training Accuracy : 0.884165346622467
Testing Precision: 0.009955354034900665
Testing Accuracy: 0.7568089365959167
